In [1]:
import sys
sys.path.append('../')
from poker_env_multi import PokerEnvMulti
from agents.random_policy import RandomActions
from agents.heuristic_policy import HeuristicPolicy
from ray.rllib.algorithms.ppo import PPOConfig
from gym import spaces
import mpu
import numpy as np
import ray

from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMING is dep

In [2]:
def select_policy(agent_id, episode, **kwargs):
    if agent_id == "player_0":
        return "random"
    elif agent_id == "player_1":
        return "Heuristic_10"
    elif agent_id == "player_2":
        return "Heuristic_100"
    elif agent_id == "player_3":
        return "Heuristic_1000"
    return "Heuristic_1000"

def env_creator(config):
    env = PokerEnvMulti(select_policy, config)
    return env

register_env("poker", lambda config: env_creator(config))

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
heuristic_observation_space = spaces.Dict({
            "hand": spaces.Box(0, 1, shape=(28, )),
            "community": spaces.Box(0, 1, shape=(28, ))
        })
action_space = spaces.Discrete(3)

config = (
    PPOConfig()
    .rollouts(
        num_rollout_workers=0,
        num_envs_per_worker=1,
    )
    .training(
        train_batch_size=128,
        gamma=0.99,
    )
    .multi_agent(
        policies={
           "random": PolicySpec(policy_class=RandomActions),
           "Heuristic_10": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 0}),
           "Heuristic_100": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 1}),
           "Heuristic_1000": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 2}),
            #"learned": PolicySpec(
            #    config={
            #        "model": {"use_lstm": use_lstm},
            #        "framework": args.framework,
            #    }
            #),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=[],
    )
    .resources(num_gpus=0)\
)
trainer = config.build(env="poker")


2022-11-07 16:38:09,286	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.
2022-11-07 16:38:09,288	WARNING ppo.py:350 -- `train_batch_size` (128) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 128.
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ray/_private/ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.1/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-11-07 16:38:11,758	INFO worker.py:1518 -- Started a local Ray instance.
2022-11-07 16:38:13,857	WARNING env.p

In [4]:
trainer.train()

progress
progress
progress
progress
showdown
done
{'hands': ['KING of CLUBS; KING of HEARTS; ', 'TWO of DIAMONDS; THREE of CLUBS; ', 'TWO of HEARTS; FOUR of DIAMONDS; ', 'ACE of SPADES; THREE of SPADES; '], 'community': 'JACK of DIAMONDS; ACE of HEARTS; JACK of SPADES; TWO of SPADES; THREE of HEARTS; ', 'scores': array([[0., 0., 0.],
       [2., 2., 4.],
       [4., 4., 4.],
       [0., 0., 0.]]), 'winners': 2, 'chips': [98, 96, 109, 97]}
progress
progress
progress
progress
showdown
done
{'hands': ['ACE of SPADES; FOUR of HEARTS; ', 'JACK of SPADES; JACK of HEARTS; ', 'ACE of DIAMONDS; QUEEN of SPADES; ', 'TWO of DIAMONDS; KING of SPADES; '], 'community': 'QUEEN of HEARTS; THREE of SPADES; KING of HEARTS; QUEEN of DIAMONDS; JACK of DIAMONDS; ', 'scores': array([[4., 6., 6.],
       [6., 4., 6.],
       [0., 0., 0.],
       [0., 0., 0.]]), 'winners': 1, 'chips': [93, 104, 108, 95]}
progress
progress
progress
progress
showdown
done
{'hands': ['KING of DIAMONDS; JACK of HEARTS; ', 'KING o

{'custom_metrics': {},
 'episode_media': {},
 'num_recreated_workers': 0,
 'info': {'learner': {},
  'num_env_steps_sampled': 128,
  'num_env_steps_trained': 128,
  'num_agent_steps_sampled': 125,
  'num_agent_steps_trained': 125},
 'sampler_results': {'episode_reward_max': 5.0,
  'episode_reward_min': -3.0,
  'episode_reward_mean': 1.5,
  'episode_len_mean': 20.166666666666668,
  'episode_media': {},
  'episodes_this_iter': 6,
  'policy_reward_min': {'Heuristic_1000': -8.0},
  'policy_reward_max': {'Heuristic_1000': 15.0},
  'policy_reward_mean': {'Heuristic_1000': 0.375},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [0.0, 5.0, -3.0, 1.0, 1.0, 5.0],
   'episode_lengths': [20, 15, 14, 37, 19, 16],
   'policy_Heuristic_1000_reward': [-4.0,
    -4.0,
    13.0,
    -5.0,
    13.0,
    -1.0,
    -2.0,
    -5.0,
    -2.0,
    -4.0,
    7.0,
    -4.0,
    -3.0,
    5.0,
    5.0,
    -6.0,
    -4.0,
    -8.0,
    -2.0,
    15.0,
    15.0,
    -5.0,
    -5.0,
    0.0]},
  'sample